In [1]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
import warnings

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df1 = pd.read_csv('bank_statement.csv')
df2 = pd.read_csv('checkout.csv')

In [3]:
# Remove special character

df1.desc = df1.desc.apply(lambda x: re.sub(r"[^a-zA-Z0-9]+",' ',x))
df2.buyer_name = df2.buyer_name.apply(lambda x: re.sub(r"[^a-zA-Z0-9]+",' ',x))
df1.desc = df1.desc.apply(lambda x: re.sub(r"TRANSFER",'',x))
df2.buyer_name = df2.buyer_name.apply(lambda x: re.sub(r"TRANSFER",'',x))

In [4]:
# Sorting amount by value
df1 = df1.sort_values(by=['stmt_amount'])
df2 = df2.sort_values(by=['ckt_amount'])

# Converting to set
df1.desc = df1.desc.apply(lambda x : set(x.split()))
df2.buyer_name = df2.buyer_name.apply(lambda x : set(x.split()))

### Matching Price and Description

In [5]:
prices = sorted(list(set(df2.ckt_amount))) # using set for non repeat values

1. Go through unique prices one by one
2. For each price, query all the bank statements and transactions
3. For each transaction, find a bank statements that matches best
4. If the transaction not found, will be processed later with fuzzy

In [6]:
# Matching prices and description

bs, co = df1.values.tolist(), df2.values.tolist()
curr_bank, curr_trans, pend_bank, pend_trans = [], [], [], []
answer = []
no_ans_trans, no_ans_bank = [], []

for price in prices:
    while bs and bs[0][1] == price:
        curr_bank.append(bs.pop(0))
    while co and co[0][1] == price:
        curr_trans.append(co.pop(0))
        
    for trans in curr_trans:
        found = False
        curname = trans[2]
        bk = list(filter(lambda x : x[2].intersection(curname), curr_bank))
        if bk: 
            b = max(bk, key=lambda x : len(x[2].intersection(curname)))
            answer.append((trans,b))
            curr_bank.remove(b)
        else: 
            no_ans_trans.append(trans)
            
    no_ans_bank.extend(curr_bank)
        
    curr_bank, curr_trans = [], []

In [7]:
len(no_ans_bank)

3143

In [8]:
len(no_ans_trans)

3143

### Improving with Fuzzy Search

In [9]:
# Getting IDs of transaction that does not match
no_id_bank, no_id_trans = [], []
for x in range(len(no_ans_bank)):
    no_id_bank.append(no_ans_bank[x][0])
    no_id_trans.append(no_ans_trans[x][0])

In [10]:
# Getting subset of dataframe that previously has no match
checkout2 = df2[df2['ckt_id'].isin(no_id_trans)]
bank_statement2 = df1[df1['stmt_id'].isin(no_id_bank)]

In [11]:
# Converting Set to string
bank_statement2['desc'] = bank_statement2['desc'].apply(lambda x: ' '.join(sorted(list(x))))
checkout2['buyer_name'] = checkout2['buyer_name'].apply(lambda x: ' '.join(sorted(list(x))))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
prices = sorted(list(set(checkout2.ckt_amount)))

In [13]:
# Applying Fuzzy Search
bs2, co2 = bank_statement2.values.tolist(), checkout2.values.tolist(), 
curr_bank2, curr_trans2 = [], []
answer2 = []
for price in prices:
    while bs2 and bs2[0][1] == price:
        curr_bank2.append(bs2.pop(0))
    while co2 and co2[0][1] == price:
        curr_trans2.append(co2.pop(0))
    for trans in curr_trans2:
        curname = trans[2]
        b = max(curr_bank2, key= lambda x: fuzz.partial_ratio(curname, x[2]))
        answer2.append((trans, b))
        curr_bank2.remove(b)
    curr_bank2, curr_trans2 = [], []

In [14]:
pd.DataFrame(answer2).to_csv('analysis.csv')

In [15]:
# Construct final dataframe
fs1 = [(x[0][0], x[1][0]) for x in answer]
fs2 = [(x[0][0], x[1][0]) for x in answer2]
fs = pd.DataFrame(fs1+fs2)
fs.columns=['ckt_id', 'stmt_id']
fs=fs[['stmt_id', 'ckt_id']]

In [16]:
# Output CSV
print(fs.shape)
fs.to_csv('Final_Answer.csv', index=False)

(240000, 2)
